In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import torch.nn.functional as F

import copy
import random
from numpy import linalg as LA
import numpy as np

In [2]:
from torchdiffeq import odeint_adjoint as odeint

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler

In [155]:
def ising_to_qubo(n, J, h):
    # Randomly generate adjacency matrix with 50% chance of each entry being 1
    adj_matrix = np.random.randint(0, 2, size=(n, n))  # This line replaces the first nested loop

    # Create a diagonal matrix with all diagonal entries equal to h
    diagonal_matrix = np.eye(n) * h

    # Create the off-diagonal entries based on J and adj_matrix
    off_diagonal_matrix = J * adj_matrix
    np.fill_diagonal(off_diagonal_matrix, 0)  # Set diagonal entries to 0

    # Sum the diagonal and off-diagonal matrices to get the final qubo_matrix
    qubo_matrix = diagonal_matrix + off_diagonal_matrix

    return qubo_matrix

In [156]:
from torchdyn.models import NeuralODE

# Define the continuous dynamics for the Neural ODE
class ContinuousDynamics(nn.Module):
    def __init__(self, input_size):
        super(ContinuousDynamics, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, input_size)

    def forward(self, t, x, args=None):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)
    
class RecursiveNN(nn.Module):
    def __init__(self, input_size, output_size, iterations):
        super(RecursiveNN, self).__init__()
        self.iterations = iterations
        self.fc = nn.Linear(input_size, output_size)
        self.normalization = nn.LayerNorm(output_size)
        self.skip_fc = nn.Linear(input_size, output_size)  # skip connection

    def forward(self, x):
        outputs = []
        for _ in range(self.iterations):
            residual = x  # preserve the input for skip connection
            x = F.leaky_relu(self.fc(x))  # using LeakyReLU
            x = self.normalization(x) + self.skip_fc(residual)  # added skip connection
            outputs.append(x)
        return torch.stack(outputs)  # Stack outputs to ensure a consistent tensor shape across batches

class CustomLoss(nn.Module):
    def __init__(self, n, l1_coef=1e-4):  # 1e-4 is the L1 regularization coefficient
        super(CustomLoss, self).__init__()
        self.n = n
        self.l1_coef = l1_coef

    def forward(self, outputs, target):
        total_loss = 0
        weight = 1.0
        for output in torch.unbind(outputs, dim=0):  # Unbind along the iteration dimension
            reshaped_target = target.view(-1, self.n, self.n)
            reshaped_output = torch.sin(output).view(-1, self.n, 1)  # Reshape output to [64, 16, 1]
            product = reshaped_target * reshaped_output  # Broadcasting will expand the last dimension of output
            total_loss += weight * torch.norm(product)
            weight *= 0.9
        
        l1_reg = sum(p.abs().sum() for p in model.parameters())  # L1 regularization
        total_loss += self.l1_coef * l1_reg
        
        return total_loss

In [174]:
samples = 10000
i = 64 * 64
o = 64

x = torch.randn(samples, o) + 1.0j * torch.randn(samples, o)  # Pre-compute random values

In [175]:
# Pre-compute random values
random_values1 = ((-1)**(np.random.randint(0, 100, size=samples) % 2) * np.random.random(samples))
random_values2 = ((-1)**(np.random.randint(0, 100, size=samples) % 2) * np.random.random(samples))

# Initialize y
y = np.empty((samples, o*o))

# Compute input_mat and update y and x in a loop (if vectorization of ising_to_qubo is not possible)
for idx in range(samples):
    input_mat = ising_to_qubo(o, random_values1[idx], random_values2[idx])
    y[idx] = input_mat.flatten()

# Normalize and compute angle of x
norms = np.linalg.norm(x, axis=1, keepdims=True)
x_normalized = x / norms
x_angle = np.angle(x_normalized)

# Convert to torch tensors
y_tensor = torch.from_numpy(y)
x_tensor = torch.from_numpy(x_angle)

In [176]:
print(x_tensor)

tensor([[ 1.4912, -0.1094,  0.9030,  ...,  0.0578,  2.5329, -0.0316],
        [-2.6746,  1.9726,  1.7977,  ...,  3.1094, -0.1936,  2.6898],
        [-0.9938,  2.9547,  3.1292,  ..., -1.6752,  1.5901, -1.6767],
        ...,
        [-1.7223,  0.4721, -0.8184,  ...,  0.0784, -0.9643, -0.7622],
        [-3.0673,  0.1923,  2.1269,  ...,  0.3261,  2.8921,  1.0635],
        [ 1.3754, -0.1616,  3.1067,  ..., -0.9389,  2.1736,  1.5637]])


In [177]:
from torch.utils.data import DataLoader, TensorDataset
train_size = int(0.8 * samples)
val_size = samples - train_size
train_dataset, val_dataset = torch.utils.data.random_split(TensorDataset(x_tensor, y_tensor), [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

In [178]:
# Instantiate the model, loss function, and optimizer
model = RecursiveNN(input_size=o, output_size=o, iterations=50)
criterion = CustomLoss(o)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)  # 1e-4 is the L2 regularization coefficient
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [ ]:
epochs = 100

# Training loop
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    train_loss = 0.0
    scheduler.step()  # Step the learning rate scheduler
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # gradient clipping
        optimizer.step()
        train_loss += loss.item()  # Accumulate training loss

    # Calculate the average training loss for this epoch
    avg_train_loss = train_loss / len(train_loader)
    
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():  # No gradient computation during validation
        for batch_x, batch_y in val_loader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()  # Accumulate validation loss
    
    # Calculate the average validation loss for this epoch
    avg_val_loss = val_loss / len(val_loader)
    
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')

Epoch 1/100, Training Loss: 1696.9318, Validation Loss: 780.3691
Epoch 2/100, Training Loss: 590.6208, Validation Loss: 440.6875
Epoch 3/100, Training Loss: 388.4503, Validation Loss: 336.8192
Epoch 4/100, Training Loss: 313.5949, Validation Loss: 288.9871
Epoch 5/100, Training Loss: 273.7420, Validation Loss: 252.5974
Epoch 6/100, Training Loss: 243.9742, Validation Loss: 228.9491
